# Process BenchExec Experiments

This notebook post-processes the results obtained from [Benchexec](https://github.com/sosy-lab/benchexec) tables.

The Benchexec tables contain sets of (repeated) stat columns, one per run set. Each run set can represent a specific solver. Each row is a problem instance, called a _task_ in Benchexec.

This set-up is not very useful to do plotting or stat tables. Instead, we would like to re-shape and have a special column that records the solver, and  so one set of stat columns (with no repetition).

This network will then produce two tables:

1. A flat table of stats, that can be used for further analysis and plotting, with the solver being recorded in a new column.
2. A coverage table per domain and solver, typically reported in papers.

Sebastian Sardina 2024 - ssardina@gmail.com

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import glob
import re

# sys.dont_write_bytecode = True  # prevent creation of .pyc files

CSV_FOLDER = "stats/ecai23-redo-benchexec-jul24/"
CSV_FOLDER = "stats/cfondasp-miner-small/"
NAME_EXPERIMENT = "cfond"

## 1. Flatten Benchexec CSV tables

Collect all CSV benchexec table result files under `CSV_FOLDER` folder.

In [13]:
# files produced when tables come from single benchexec run (one .xml.bz2 file)
benchexec_csv_files = glob.glob(os.path.join(CSV_FOLDER, "**", "benchmark-*.csv"), recursive=True)

# files produced when tables come from multiple benchexec runs (many .xml.bz2 files)
benchexec_csv_files += glob.glob(
    os.path.join(CSV_FOLDER, "**", "results.*.table.csv"), recursive=True
)

print(f"CSV benchexec table files found to extract and combine stats in folder {CSV_FOLDER}:")
benchexec_csv_files

CSV benchexec table files found to extract and combine stats in folder stats/cfondasp-miner-small/:


['stats/cfondasp-miner-small/results/results.2025-02-09_20-47-17.table.csv']

Next, **load all CSV files into a single dataframe**. As explained above, each CSV file may include results over many _run sets_, with each having its own (set of) columns.

Each row is the result of a _task_ in the experiment, and every run set has its columns stats for such task. Presumably sets of columns share the same schema.

We then need reshape this structure to have just one set of columns and a new column identifying the run set. So each original row will be expanded into many rows, one per run set.

The first three lines contain header:

1. First line contains the **_tool_** used. It starts with `tool` followed by the name of the tool repeated multiple times (to match no. of columns).
2. Second line contains the **_runs_** of the experiment. It starts with `run set` and then sets of columns with the name of the runs.
3. Third line contains the **_stats_** column names repeated per run set. First column is for name of the task.

In [14]:
RENAME_COLS = {"benchmarks/benchexe/tasks/": "id", "cputime (s)": "cputime", "walltime (s)": "walltime", "memory (MB)": "memory_mb"}

def get_meta_csv(file):
    """Given a benchexec CSV file, extract the runs (e.g., prp, prp_inv) and how many columns per run"""
    with open(file, "r") as f:
        # first line contains the tool used (repeated one per column needed); e.g.,  PP-FOND
        tools_header = f.readline().split()[1:]
        # second line contains the run/solvers used in the experiment (e.g., prp, prp_inv) and starts with "run set" to be ignored - will have duplicates, one per stat col in run
        runs_header = f.readline().split()[2:]

    # get the run names (e.g., prp, prp_inv), drop duplicates but preserve order (order of columns in CSV)
    # OBS: cannot just use set() as that will change the order of columns in CSV
    runs = list(dict.fromkeys(runs_header))

    # no of stat columns per run
    no_cols = int(len(runs_header) / len(runs))

    return runs, no_cols


# iterate through each CSV file
dfs = []
for f in benchexec_csv_files:
    runs, no_cols = get_meta_csv(f)
    print(f"Runs in file {f}: {runs} with {no_cols} stat columns")

    # go over each set of run columns (a CSV table file may contain several runs, each with the same columns)
    for k, r in enumerate(list(runs)):
        col_idx = [0] + list(range(k*no_cols + 1, k*no_cols + no_cols + 1))
        print(f"\t Extracting run '{r}' in columns: {col_idx}")

        # read the CSV file from line 3+ (line 3 is header)
        df = pd.read_csv(f, delimiter="\t", skiprows=2, usecols=col_idx)
        df.rename(columns=lambda x: x.split('.')[0], inplace=True)

        df.columns.values[0] = "task"
        # df.rename(columns={df.columns[1]: "task"})

        # populate column run with name of run-solver r
        df.insert(1, 'run', r)
        dfs.append(df)

df_csv = pd.concat(dfs).reset_index(drop=True)

df_csv.rename(columns=RENAME_COLS, inplace=True)

print("Runs found:", df_csv["run"].unique())
for s in df_csv["run"].unique():
    print(s)
# df.set_index("task", inplace=True)


# df_csv.query("task == 'acrobatics_01.yml' and run == 'cfondasp1-reg.FOND'")
# df_csv.query("run == 'cfondasp2-reg.FOND'")
df_csv

Runs in file stats/cfondasp-miner-small/results/results.2025-02-09_20-47-17.table.csv: ['cfondasp-fsat.MINER-SMALL', 'cfondasp-reg.MINER-SMALL'] with 6 stat columns
	 Extracting run 'cfondasp-fsat.MINER-SMALL' in columns: [0, 1, 2, 3, 4, 5, 6]
	 Extracting run 'cfondasp-reg.MINER-SMALL' in columns: [0, 7, 8, 9, 10, 11, 12]
Runs found: ['cfondasp-fsat.MINER-SMALL' 'cfondasp-reg.MINER-SMALL']
cfondasp-fsat.MINER-SMALL
cfondasp-reg.MINER-SMALL


,task,run,status,cputime,walltime,memory_mb,planner_time,policy_size
0,miner_01.yml,cfondasp-fsat.MINER-SMALL,True,10.221408,9.274593,239.194112,6.419276,18
1,miner_02.yml,cfondasp-fsat.MINER-SMALL,True,8.672183,8.143851,237.326336,5.244447,15
2,miner_03.yml,cfondasp-fsat.MINER-SMALL,True,9.991114,9.451395,267.620352,6.565095,14
3,miner_04.yml,cfondasp-fsat.MINER-SMALL,True,29.911280,26.532475,482.852864,23.646294,17
4,miner_05.yml,cfondasp-fsat.MINER-SMALL,True,34.188173,29.191234,439.996416,26.374557,19
5,miner_06.yml,cfondasp-fsat.MINER-SMALL,True,36.875697,31.872171,539.783168,29.065801,18
6,miner_07.yml,cfondasp-fsat.MINER-SMALL,True,67.825356,56.313294,704.856064,53.415865,20
7,miner_08.yml,cfondasp-fsat.MINER-SMALL,True,43.557440,38.233510,628.289536,35.444397,18
8,miner_09.yml,cfondasp-fsat.MINER-SMALL,True,58.068282,49.650738,730.161152,47.696455,19
9,miner_01.yml,cfondasp-reg.MINER-SMALL,True,8.785078,6.861987,193.794048,4.684582,18


We next **enrich** the dataframe with the following derived columns:

* domain
* instance
* solver
* solved (boolean)

In [16]:
def get_benchmark_labels(task_name):
    """From the task description name (e.g., acrobatics_01.yml), extract the benchmark labels, like domain, instance"""
    regex = r"(.+)_([0-9]+)\.yml"

    match = re.match(regex, task_name)
    if match:
        # print(match.groups())
        domain = match.group(1)
        instance = match.group(2)
    else:
        print("Problem extracting labels from task name", task_name)
    return domain, instance

df = df_csv.copy()

# 1 - split task name into domain and instances
df["benchmark"] = df.reset_index()["task"].map(get_benchmark_labels).values
df["domain"] = df["benchmark"].str.get(0)
df["instance"] = df["benchmark"].str.get(1)
df.drop(columns=["benchmark"], inplace=True)

# 2 - map status from benchexec to integers status
map_status = {
    "true": 1,
    "false": 0,
    "True": 1,
    "False": 0,
    False: 0,
    True: 1,
    "OUT OF MEMORY (false)": -2,
    "TIMEOUT (false)": -1,
    "TIMEOUT (true)": 1,
}
df["status2"] = df["status"].map(map_status)

missing_mapping = df[df["status2"].isnull()].shape[0]
if missing_mapping > 0:
    missing_status = [x for x in df["status"].unique() if x not in map_status.keys()]
    print(f"WARNING: {missing_mapping} status values not mapped:", missing_status)
    print(df[df["status2"].isnull()])

df["status"] = df["status2"]
df.drop(columns=["status2"], inplace=True)

# 3 - define Boolean column solved to flag if solved or not based on status
df.insert(3, "solved", df["status"].apply(lambda x: True if x == 1 else False))


# 4 - extract solver from run name
def map_solver(run):
    MAP = {
        "prp.FOND": "PRP",
        "paladinus.FOND": "PAL",
        "fondsat-glucose.FOND": "FSAT-GL",
        "fondsat-minisat.FOND": "FSAT-MS",
        "cfondasp1-reg.FOND": "ASP1-reg",
        "cfondasp1-fsat.FOND": "ASP1-fsat",
        "cfondasp2-reg.FOND": "ASP2-reg",
        "cfondasp2-fsat.FOND": "ASP2-fsat",
        "cfondasp-reg.FOND": "ASP-reg",
        "cfondasp-fsat.FOND": "ASP-fsat"
    }
    if run in MAP:
        return MAP[run]
    else:
        if "cfondasp-reg" in run:
            return "ASP-reg"
        elif "cfondasp-fsat" in run:
            return "ASP-fsat"
        return np.nan

df["solver"] = df["run"].map(map_solver)

df["solver"] = df["run"].apply(lambda x: map_solver(x))


print("Domains:", df["domain"].unique())
print("Solvers:", df["solver"].unique())

# sanity check status
# df.query("status not in [-1,0,-2,1]")
# df.status = df.status.astype(int) # convert to int
# df.loc[df.status == "OUT OF MEMORY (false)"]
# df.loc[df.status == -1]

# df.dtypes

# note that status should be integer; if float it is bc there must be NaN value!
df

Domains: ['miner']
Solvers: ['ASP-fsat' 'ASP-reg']


,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,miner_01.yml,cfondasp-fsat.MINER-SMALL,1,True,10.221408,9.274593,239.194112,6.419276,18,miner,01,ASP-fsat
1,miner_02.yml,cfondasp-fsat.MINER-SMALL,1,True,8.672183,8.143851,237.326336,5.244447,15,miner,02,ASP-fsat
2,miner_03.yml,cfondasp-fsat.MINER-SMALL,1,True,9.991114,9.451395,267.620352,6.565095,14,miner,03,ASP-fsat
3,miner_04.yml,cfondasp-fsat.MINER-SMALL,1,True,29.911280,26.532475,482.852864,23.646294,17,miner,04,ASP-fsat
4,miner_05.yml,cfondasp-fsat.MINER-SMALL,1,True,34.188173,29.191234,439.996416,26.374557,19,miner,05,ASP-fsat
5,miner_06.yml,cfondasp-fsat.MINER-SMALL,1,True,36.875697,31.872171,539.783168,29.065801,18,miner,06,ASP-fsat
6,miner_07.yml,cfondasp-fsat.MINER-SMALL,1,True,67.825356,56.313294,704.856064,53.415865,20,miner,07,ASP-fsat
7,miner_08.yml,cfondasp-fsat.MINER-SMALL,1,True,43.557440,38.233510,628.289536,35.444397,18,miner,08,ASP-fsat
8,miner_09.yml,cfondasp-fsat.MINER-SMALL,1,True,58.068282,49.650738,730.161152,47.696455,19,miner,09,ASP-fsat
9,miner_01.yml,cfondasp-reg.MINER-SMALL,1,True,8.785078,6.861987,193.794048,4.684582,18,miner,01,ASP-reg


Analyze particular cases:

In [ ]:
df.query('solver == "ASP2-reg" and domain == "blocksworld-ipc08"')

Check that solver has been fully extracted (no nulls):

In [17]:
df.query("solver.isnull()")

,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver


Finally, save all results into a complete CSV file. This file can be later used to plot time-coverage graphs as in Nitin's R script.

These tables are not flatten, runs are not across column sets but there is a designated column `solver` that specifies the run of the row.

In [18]:
df.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_benchexec_stats.csv"), index=False)

## 2. Coverage Analysis and Table

We now generate **coverage** tables, as they often appear in papers. Basically, we compute the following per domain and solver-run:

- **Coverage:** % of solved instances solved by the solver-run; and
- **Stat metrics:** mean on time, memory usage, and policy size.

In [19]:
print(df.shape)
df.head()

(18, 12)


,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,miner_01.yml,cfondasp-fsat.MINER-SMALL,1,True,10.221408,9.274593,239.194112,6.419276,18,miner,01,ASP-fsat
1,miner_02.yml,cfondasp-fsat.MINER-SMALL,1,True,8.672183,8.143851,237.326336,5.244447,15,miner,02,ASP-fsat
2,miner_03.yml,cfondasp-fsat.MINER-SMALL,1,True,9.991114,9.451395,267.620352,6.565095,14,miner,03,ASP-fsat
3,miner_04.yml,cfondasp-fsat.MINER-SMALL,1,True,29.911280,26.532475,482.852864,23.646294,17,miner,04,ASP-fsat
4,miner_05.yml,cfondasp-fsat.MINER-SMALL,1,True,34.188173,29.191234,439.996416,26.374557,19,miner,05,ASP-fsat


Calculate % ratio per set/domain/sub_domain/run-solver.

In [20]:
df_grouped = df.groupby(["domain", "solver"])

#   df_grouped.sum()[["solved"]] = sum all the True instances (sum over bool = number of True)
#   df_grouped.count()[["solved"]] = number of rows in solved column (includes True and False values)
df_coverage = (
    df_grouped.sum(numeric_only=True)[["solved"]] / df_grouped.count()[["solved"]]
)
df_coverage

solved
domain solver          
miner  ASP-fsat     1.0
       ASP-reg      1.0

Calculate mean metric (for CPU time, memory, and policy size) across the solved instances.

In [21]:
columns = ["domain", "solver", "cputime", "memory_mb", "policy_size"]
df_solved = df.query("solved == True")[columns]

df_solved_grouped = df_solved.groupby(["domain", "solver"])
df_metrics = df_solved_grouped.mean()
df_metrics

cputime   memory_mb  policy_size
domain solver                                      
miner  ASP-fsat  33.256770  474.453333    17.555556
       ASP-reg   36.146694  346.352299    17.555556

Put together **Coverage** and **Metrics** tables.

In [25]:
column_names = {
    "solved": "cov",
    "cputime": "time",
    "memory_mb": "mem",
    "policy_size": "size",
}

df_stats = df_coverage.join(df_metrics, how="inner")
df_stats.rename(columns=column_names, inplace=True)

df_stats = df_stats.reset_index()

df_stats

# df_stats.query('domain == "blocksworld-ipc08"')

,domain,solver,cov,time,mem,size
0,miner,ASP-fsat,1.0,33.256770,474.453333,17.555556
1,miner,ASP-reg,1.0,36.146694,346.352299,17.555556


Finally, pivot the column `solver` into (set of) columns, one per solver.

In [23]:
df_stats_pivot = df_stats.pivot(
    index=["domain"],
    values=["cov", "time", "mem", "size"],
    columns="solver",
)
df_stats_pivot.reset_index(
    inplace=True
)  # unfold multi-index into columns (create integer index)
df_stats_pivot.columns = [
    "_".join(tup).rstrip("_") for tup in df_stats_pivot.columns.values
]

# flat index, but multi-column: 1. coverage / time / policy size and 2. each solver/run
df_stats_pivot = df_stats_pivot.round(2)

df_stats_pivot

,domain,cov_ASP-fsat,cov_ASP-reg,time_ASP-fsat,time_ASP-reg,mem_ASP-fsat,mem_ASP-reg,size_ASP-fsat,size_ASP-reg
0,miner,1.0,1.0,33.26,36.15,474.45,346.35,17.56,17.56


Save coverage table to a CSV file, this can be used in papers.

In [24]:
df_stats_pivot.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_coverage_table.csv"), index=False)